## Реализация собственного нейросетевого пакета для запуска и обучения нейронных сетей

Задание состоит из трёх частей:
1. Реализация прямого вывода нейронной сети (первое практическое задание)
2. Реализация градиентов по входу и распространения градиента по сети (back propagation)
3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

###  1. Реализация вывода собственной нейронной сети

1.1 Внимательно ознакомьтесь с интерфейсом слоя. Любой слой должен содержать как минимум три метода:
- конструктор
- прямой вывод 
- обратный вывод, производные по входу и по параметрам

In [2]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'       
    def forward(self, input_data):
        pass
    def backward(self, input_data):
        return [self.grad_x(input_data), self.grad_param(input_data)]
    
    def grad_x(self, input_data):
        pass
    def grad_param(self, input_data):
        return []
    
    def update_param(self, grads, learning_rate):
        pass


1.2 Ниже предствален интерфейс класса  Network. Обратите внимание на реализацию метода predict, который последовательно обрабатывает входные данные слой за слоем.

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class Network(object):
    def __init__(self, layers, loss=None, debag_print=None):
        self.debag_print = debag_print
        self.name = 'Network'
        self.layers = layers
        self.loss = loss
        self.forward_data_layers = []
        self.grad_layers = []
        
    def forward(self, input_data):
        return self.predict(input_data)
    
    def grad_x(self, input_data, labels):
        self.forward_data_layers = []
        self.grad_layers = []
        current_input = np.copy(input_data)
        self.forward_data_layers.append(current_input)       
        for layer in self.layers:
             current_input = layer.forward(current_input) 
             self.forward_data_layers.append(current_input)

        prev_grad = self.loss.grad_x(self.forward_data_layers[-1], labels)
        length = len(self.layers)
        
        for i in range(length):
            current_grad = self.layers[length - (i+1)].grad_x(self.forward_data_layers[length - (i+1)])
            if  self.debag_print == 1:
                print('prev')
                print(np.shape(np.asarray(prev_grad)))
            prev_grad = np.expand_dims(prev_grad, axis=1)
            if  self.debag_print == 1:
                print('prev_expanded')
                print(np.shape(prev_grad))
                print('cur')
                print(np.shape(np.asarray(current_grad)))
            prev_grad = np.matmul(prev_grad, current_grad)
            if  self.debag_print == 1:
                print('mul(prev, curr)')
                print(np.shape(prev_grad))
            prev_grad = np.squeeze(prev_grad)
            if  self.debag_print == 1:
                print('prev_squeeze')
                print(np.shape(prev_grad))
            self.grad_layers.append(prev_grad)

        return self.grad_layers[-1]

    
    def grad_param(self, input_data, labels):
        self.grad_x(input_data, labels)
        grad_list = []      
        length = len(self.layers)
        for i in range(length):
            temp= []
            layer = self.layers[i]
            if layer.name == 'Dense':
                curr = layer.grad_param(self.forward_data_layers[i])
                for j in range(2):
                    curr_param = self.grad_layers[length - i - 2]
                    curr_param = np.expand_dims(curr_param, axis=1)
                    curr_param = np.matmul(curr_param, curr[j])
                    curr_param = np.squeeze(curr_param)
                    temp.append(curr_param)

                grad_list.append(temp)
                    
        return grad_list
        
    def update(self, grad_list, learning_rate):
        count = 0
        for layer in self.layers:
            if layer.name == "Dense":
                layer.update_param(grad_list[count], learning_rate)
                count +=1
       
    
    def predict(self, input_data):
        current_input = input_data
        for layer in self.layers:
            current_input = layer.forward(current_input)     
        return current_input
    
    def calculate_loss(self, input_data, labels):
        return self.loss.forward(self.predict(input_data), labels)
    
    def train_step(self, input_data, labels, learning_rate=0.001):
        grad_list = self.grad_param(input_data, labels)
        self.update(grad_list, learning_rate)
    
    
    def fit(self, trainX, trainY, validation_split=0.25, 
            batch_size=1, nb_epoch=1, learning_rate=0.01):
  
        train_x, val_x, train_y, val_y = train_test_split(trainX, trainY, 
                                                          test_size=validation_split,
                                                          random_state=42)
        for epoch in range(nb_epoch):
            #train one epoch
            for i in tqdm(range(int(len(train_x)/batch_size))):
                batch_x = train_x[i*batch_size: (i+1)*batch_size]
                batch_y = train_y[i*batch_size: (i+1)*batch_size]
                self.train_step(batch_x, batch_y, learning_rate)
            #validate
            val_accuracy = self.evaluate(val_x, val_y)
            print('%d epoch: val %.2f' %(epoch+1, val_accuracy))
            
    def evaluate(self, testX, testY):
        y_pred = np.argmax(self.predict(testX), axis=1)            
        y_true = np.argmax(testY, axis=1)
        val_accuracy = np.sum((y_pred == y_true))/(len(y_true))
        return val_accuracy

#### 1.1 (6 баллов) Необходимо реализовать метод forward для вычисления следующих слоёв:

- DenseLayer
- ReLU
- Softmax
- FlattenLayer
- MaxPooling

In [4]:
#импорты
import numpy as np

In [5]:
import re


class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
        self.name = 'Dense'
        self.input_dim = input_dim
        self.output_dim = output_dim
        if W_init is None or b_init is None:
            self.W = np.random.uniform(-np.sqrt(3/input_dim), np.sqrt(3/input_dim), size=(input_dim, output_dim))
            self.b = np.zeros(output_dim, 'float32')
        else:
            self.W = W_init
            self.b = b_init
    def forward(self, input_data):
        return np.dot(input_data, self.W)+self.b

    def grad_x(self, input_data):
        return self.W.T

    def grad_b(self, input_data):
        res = np.zeros(shape=(input_data.shape[0], self.output_dim, self.output_dim), )
        for i in range(input_data.shape[0]):
            res[i] = np.eye(self.output_dim)
        return res

    def grad_W(self, input_data):
        res = np.zeros(shape=(input_data.shape[0], self.output_dim, self.output_dim* self.input_dim))

        for i in range(input_data.shape[0]):
            count = 0
            for j in range(self.output_dim):  
                count_= 0 
                for k in range (self.output_dim* self.input_dim):
                    if k%self.output_dim == 0:
                        res[i, j, k+count] = input_data[i, count_]
                        count_ += 1
                count+=1
        return res
    
    def update_W(self, grad, learning_rate):
        self.W -= learning_rate * np.mean(grad, axis=0).reshape(self.W.shape)
    
    def update_b(self, grad,  learning_rate):
        self.b -= learning_rate * np.mean(grad, axis=0)
        
    def update_param(self, params_grad, learning_rate):
        self.update_W(params_grad[0], learning_rate)
        self.update_b(params_grad[1], learning_rate)
    
    def grad_param(self, input_data):
        return [self.grad_W(input_data), self.grad_b(input_data)]
    
class ReLU(Layer):
    def __init__(self):
        self.name = 'ReLU'
    def forward(self, input_data):
        return np.maximum(0,input_data)
    def grad_x(self, input_data):
        batch, len = input_data.shape
        out = np.zeros((batch,len, len))
        for i in range(batch):
            ans = input_data[i]*np.eye(len)
            ans[ans>0] = 1
            ans[ans<=0] = 0
            out[i] = ans
        return out
    
    
class Softmax(Layer):
    def __init__(self):
        self.name = 'Softmax'
    def forward(self, input_data):
        x = np.exp(input_data-np.max(input_data))
        for i in range(x.shape[0]):
            k = np.sum(x[i])
            for j in range(x.shape[1]):
                x[i][j] = x[i][j]/k
        return x
    def grad_x(self, input_data):
        batch , len = input_data.shape
        out = np.zeros((batch, len, len))
        forward = self.forward(input_data)
        count = 0
        for i in forward:
            out[count,:,:] =  i.reshape(1, len)*np.eye(len) -  i.reshape(1, len)* i.reshape(1, len).T
            count += 1
        return out
    


class FlattenLayer(Layer):
    def __init__(self):
        self.name = 'Flatten'
        
    def forward(self, input_data):
        return input_data.reshape(input_data.shape[0], input_data.shape[1]*input_data.shape[2]*input_data.shape[3])

    def grad_x(self):
        pass

class MaxPooling(Layer):
    def __init__(self, pool_size=(2, 2), strides=2):
        self.name = 'MaxPooling'
        self.pool_size = pool_size
        self.strides = 2
    def forward(self, input_data):
        b, h_prev, w_prev, n_c = input_data.shape
        print(h_prev)
        s = self.strides
        f = self.pool_size[0]
        h = int((h_prev - f)/s)+1 
        w = int((w_prev - f)/s)+1
  
        downsampled = np.zeros((b, h, w, n_c))
 
        def out(k, i, out_y, out_x, curr_y, curr_x, inp, w_prev):
          if curr_x + f > w_prev:
            return
          downsampled[k, out_y, out_x, i] = np.max(inp[ curr_y:curr_y+f, curr_x:curr_x+f, i])
          out(k, i, out_y, out_x + 1, curr_y, curr_x + s, inp, w_prev)
 
        for k in range(b):
          inp = input_data[k]
          for i in range(n_c):
            curr_y = out_y = 0
            while curr_y + f <= h_prev:
                curr_x = out_x = 0
                out(k, i, out_y, out_x, curr_y, curr_x, inp, w_prev)
                curr_y += s
                out_y += 1
          
        return downsampled
    
    def grad_x(self):
        pass
   

#### 1.2 (3 балла) Реализуйте теперь свёртночный слой   (опционально)

In [6]:
class Conv2D(Layer):
    def __init__(self, kernel_size, input_channels, output_channels, 
                 kernels_init=None, bias_init=None):
        self.name = 'Conv2D'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.input_channels = input_channels
        self.output_channels = output_channels
        if kernels_init is None or bias_init is None:
            pass
        else:
            self.kernels = kernels_init
            self.bias = bias_init
            
    def forward(self, input_data):
        pass
    def grad_x(self):
        pass
    def grad_kernel(self):
        pass


#### 1.4 Теперь настало время теста. 
#### Если вы всё сделали правильно, то запустив следующие ячейки у вас должна появиться надпись: Test PASSED

Переходить к дальнейшим заданиям не имеем никакого смысла, пока вы не добьётесь прохождение теста
    

#### Чтение данных

In [7]:
import numpy as np
np.random.seed(123)  # for reproducibility
from keras.utils import np_utils
from keras.datasets import mnist
 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)


#### Подготовка моделей

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, Conv2D, MaxPooling2D

def get_keras_model():
    input_image = Input(shape=(28, 28, 1))
    pool1 = MaxPooling2D(pool_size=(2,2), data_format='channels_last')(input_image)
    flatten = Flatten()(pool1)
    dense1 = Dense(10, activation='softmax')(flatten)
    model = Model(inputs=input_image, outputs=dense1)

    from keras.optimizers import Adam, SGD
    sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(X_train, Y_train, validation_split=0.25, 
                        batch_size=32, epochs=2, verbose=1)
    return model

In [9]:
def get_our_model(keras_model):
    maxpool = MaxPooling()
    flatten = FlattenLayer()
    dense = DenseLayer(196, 10, W_init=keras_model.get_weights()[0],
                       b_init=keras_model.get_weights()[1])
    softmax = Softmax()
    net = Network([maxpool, flatten, dense, softmax])
    return net

In [10]:
keras_model = get_keras_model()
our_model = get_our_model(keras_model)

c:\Users\AUser\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/2
1407/1407 [==============================] - 5s 3ms/step - loss: 0.5698 - accuracy: 0.8481 - val_loss: 0.3808 - val_accuracy: 0.8919
Epoch 2/2
1407/1407 [==============================] - 3s 2ms/step - loss: 0.3765 - accuracy: 0.8926 - val_loss: 0.3448 - val_accuracy: 0.9029


In [11]:
keras_prediction = keras_model.predict(X_test)
our_model_prediction = our_model.predict(X_test)

313/313 [==============================] - 1s 2ms/step
28


In [12]:
if np.sum(np.abs(keras_prediction - our_model_prediction)) < 0.01:
    print('Test PASSED')
else:
    print('Something went wrong!')

Test PASSED


### 2. Вычисление производных по входу для слоёв нейронной сети

#### 2.1 (1 балл) Реализуйте метод forward для класса CrossEntropy
Напоминание: $$ crossentropy = L(p, y) =  - \sum\limits_i y_i log p_i, $$
где вектор $(p_1, ..., p_k) $ -  выход классификационного алгоритма, а $(y_1,..., y_k)$ - правильные метки класса в унарной кодировке (one-hot encoding)

In [13]:
class CrossEntropy(object):
    def __init__(self, eps=0.00001):
        self.name = 'CrossEntropy'
        self.eps = eps
    
    def forward(self, input_data, labels):
        return -np.sum(labels*np.log(input_data), axis=1)
    
    def calculate_loss(self,input_data, labels):
        return self.forward(input_data, labels)
    
    def grad_x(self, input_data, lables):
        return -(lables/input_data)

#### 2.2 (2 баллa) Реализуйте метод grad_x класса CrossEntropy, который возвращает $\frac{\partial L}{\partial p}$

Проверить работоспособность кода поможет следующий тест:

In [14]:
def numerical_diff_net(net, x, labels):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (net.calculate_loss(x + delta, labels) - net.calculate_loss(x-delta, labels)) / (2*eps)
        right_answer.append(diff)
    return np.array(right_answer).T

def test_net(net):
    x = np.array([[1, 2, 3], [2, 3, 4]])
    labels = np.array([[0.3, 0.2, 0.5], [0.3, 0.2, 0.5]])
    num_grad = numerical_diff_net(net, x, labels)
    grad = net.grad_x(x, labels)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)
        
loss = CrossEntropy()
test_net(loss)

Test PASSED


#### 2.3 (2 балла)   Реализуйте метод grad_x класса Softmax, который возвращает $\frac{\partial Softmax}{\partial x}$

Проверить работоспособность кода поможет следующий тест:

In [15]:
def numerical_diff_layer(layer, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (layer.forward(x + delta) - layer.forward(x-delta)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_layer(layer):
    x = np.array([[1, 2, 3], [2, -3, 4]])
    num_grad = numerical_diff_layer(layer, x)
    grad = layer.grad_x(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)
        
layer = Softmax()
test_layer(layer)

Test PASSED


#### 2.4 (5 баллов) Реализуйте метод grad_x для классов ReLU и DenseLayer

In [16]:
layer = ReLU()
test_layer(layer)

Test PASSED


In [17]:
layer = DenseLayer(3,4)
test_layer(layer)

Test PASSED


#### 2.5 (4 балла) Для класса Network реализуйте метод grad_x, который должен реализовывать взятие производной от лосса по входу

In [18]:
net = Network([DenseLayer(3, 10), ReLU(), DenseLayer(10, 3), Softmax()], loss=CrossEntropy())
test_net(net)

Test PASSED


### 3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

#### 3.1 (4 балла) Реализуйте функции grad_b и grad_W. При подготовке теста grad_W предполагается, что W является отномерным вектором.

In [19]:
def numerical_grad_b(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(b)):
        delta = np.zeros(b.shape)
        delta[i] = eps
        dense1 = DenseLayer(input_size, output_size, W_init=W, b_init=b+delta)
        dense2 = DenseLayer(input_size, output_size, W_init=W, b_init=b-delta)
        diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_b():
    input_size = 3
    output_size = 4 
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((2, input_size))
    
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_b(x)

    num_grad = numerical_grad_b(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

test_grad_b()

Test PASSED


In [20]:
def numerical_grad_W(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            delta = np.zeros(W.shape)
            delta[i, j] = eps
            dense1 = DenseLayer(input_size, output_size, W_init=W+delta, b_init=b)
            dense2 = DenseLayer(input_size, output_size, W_init=W-delta, b_init=b)
            diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
            right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_W():
    input_size = 3
    output_size = 4 
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((4,))
    x = np.random.random((2, input_size))
        
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_W(x)

    num_grad = numerical_grad_W(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

test_grad_W()

Test PASSED


#### 3.2 (4 балла) Полностью реализуйте метод обратного распространения ошибки в функции train_step класса Network


Рекомендуем реализовать сначала функцию Network.grad_param(), которая возвращает список длиной в количество слоёв и элементом которого является список градиентов по параметрам.
После чего, имея список градиентов, написать функцию обновления параметров для каждого слоя. 

Совет: рекомендуем написать тест для кода подсчета градиента по параметрам, чтобы быть уверенным в том, что градиент через всю сеть считается правильно
    

#### 3.3 Ознакомьтесь с реализацией функции fit класса Network. Запустите обучение модели. Если всё работает правильно, то точность на валидации должна будет возрастать

In [21]:
net = Network([DenseLayer(784, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::3], Y_train[::3], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.01)

100%|██████████| 937/937 [02:05<00:00,  7.45it/s]


1 epoch: val 0.85


100%|██████████| 937/937 [02:32<00:00,  6.14it/s]


2 epoch: val 0.87


100%|██████████| 937/937 [02:07<00:00,  7.35it/s]


3 epoch: val 0.88


100%|██████████| 937/937 [02:02<00:00,  7.67it/s]


4 epoch: val 0.89


100%|██████████| 937/937 [02:01<00:00,  7.68it/s]


5 epoch: val 0.89


In [108]:
net = Network([DenseLayer(784, 20), ReLU(), DenseLayer(20, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::6], Y_train[::6], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.001)    

100%|██████████| 468/468 [03:54<00:00,  2.00it/s]


1 epoch: val 0.33


100%|██████████| 468/468 [04:01<00:00,  1.94it/s]


2 epoch: val 0.43


100%|██████████| 468/468 [03:19<00:00,  2.34it/s]


3 epoch: val 0.53


100%|██████████| 468/468 [03:07<00:00,  2.50it/s]


4 epoch: val 0.60


100%|██████████| 468/468 [03:10<00:00,  2.45it/s]

5 epoch: val 0.66


#### 3.5 (2 балла) Продемонстрируйте, что ваша реализация позволяет обучать более глубокие нейронные сети 

In [109]:
net = Network([DenseLayer(784, 50),  ReLU(), DenseLayer(50, 20),  ReLU(), DenseLayer(20, 10),  Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::6], Y_train[::6], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.001) 

100%|██████████| 468/468 [17:15<00:00,  2.21s/it]


1 epoch: val 0.22


100%|██████████| 468/468 [20:13<00:00,  2.59s/it]


2 epoch: val 0.32


100%|██████████| 468/468 [17:08<00:00,  2.20s/it]


3 epoch: val 0.40


100%|██████████| 468/468 [16:34<00:00,  2.12s/it]


4 epoch: val 0.46


100%|██████████| 468/468 [23:48<00:00,  3.05s/it]    

5 epoch: val 0.53
